<a href="https://colab.research.google.com/github/Anie0205/Langchain_With_Gemini_And_Build_RAG/blob/main/Langchain_With_Gemini_And_Build_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
! pip install -q --upgrade google-generativeai langchain-google-genai chromadb pypdf


In [41]:
from IPython.display import display
from IPython.display import Markdown
import textwrap


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [42]:
import google.generativeai as genai
from google.colab import userdata

In [43]:
import os
userdata = {
    'GOOGLE_API_KEY': '*********'
}
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

### Text Generation

In [44]:
model = genai.GenerativeModel(model_name = "gemini-pro")
model

 genai.GenerativeModel(
   model_name='models/gemini-pro',
   generation_config={}.
   safety_settings={}
)

In [45]:
response = model.generate_content("Who was Osamu Dazai?")

In [46]:
to_markdown(response.text)

> Osamu Dazai (1909-1948) was a Japanese novelist, short story writer, and essayist. He is considered one of the foremost writers of 20th-century Japanese literature. Dazai was born in Kanagi, Aomori Prefecture, Japan. He was the eleventh of fourteen children in a wealthy family. Dazai's father was a government official, and his mother was a devout Buddhist. Dazai began writing at an early age, and he published his first novel, The Captain's Daughter, in 1933. Dazai's writing is often characterized by its dark and pessimistic tone. He often wrote about themes of alienation, despair, and suicide. Dazai himself struggled with depression and addiction throughout his life, and he attempted suicide several times. Dazai's most famous novel, No Longer Human, was published in 1948. The novel is a semi-autobiographical account of the life of a young man who struggles with depression and addiction. No Longer Human was a critical and commercial success, and it is considered one of Dazai's masterpieces. Dazai died by suicide in 1948, at the age of 39. He is buried in the temple of Zuisen-ji in Mitaka, Tokyo.

### Use LangChain to Access Gemini API

In [47]:
from langchain_google_genai import ChatGoogleGenerativeAI


In [48]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY)

In [69]:
result = llm.invoke("Who was Emily Bronte?")


In [70]:
to_markdown(result.content)

> Emily Jane Brontë (30 July 1818 – 19 December 1848) was an English novelist and poet who is best known for her only novel, Wuthering Heights, now considered a classic of English literature.
> 
> She was the younger sister of Charlotte Brontë and Anne Brontë, who are also well-known novelists.

## Chat with Documents using RAG (Retreival Augment Generation)

In [51]:
!pip install langchain

In [52]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA



warnings.filterwarnings("ignore")
# restart python kernal if issues with langchain import.

In [53]:
from langchain_google_genai import ChatGoogleGenerativeAI


In [54]:
model = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY,
                             temperature=0.2,convert_system_message_to_human=True)


### RAG Pipeline: Embedding + Gemini (LLM)

In [55]:
pdf_loader = PyPDFLoader("/content/The_Art_Of_War.pdf")
pages = pdf_loader.load_and_split()
print(pages[0].page_content)

/G53/G75/G6E/G20/G54/G7A/G75/G20/G6F/G6E/G20/G74/G68/G65/G20
/G41/G72/G74/G20/G6F/G66/G20/G57/G61/G72
THE OLDEST MILITARY TREATISE IN THE WORLD
/G41/G6C/G6C/G61/G6E/G64/G61/G6C/G65 /G4F/G6E/G6C/G69/G6E/G65
/G50/G75/G62/G6C/G69/G73/G68/G69/G6E/G67/G43/G6C/G61/G73/G73/G69/G63/G20/G45/G74/G65/G78/G74/G73/G20/G53/G65/G72/G69/G65/G73


In [56]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [57]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
context = "\n\n".join(str(p.page_content) for p in pages)
texts = text_splitter.split_text(context)

In [58]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)

In [59]:
vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k":5})


In [60]:
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True

)

In [61]:
question = "What is this book about?"
result = qa_chain({"query": question})
result["result"]

'This book is about the art of war.'

In [62]:
Markdown(result["result"])

This book is about the art of war.

In [63]:
result["source_documents"]

[Document(page_content="6. Knowledge of the enemy’s dispositions can only be\nobtained from other men.\n7. Hence the use of spies, of whom there are five class-\nes: \n(1) Local spies; (2) inward spies; (3) converted spies;(4) doomed spies; (5) surviving spies.\n8.When these five kinds of spy are all at work, none\ncan discover the secret system. This is called “divinemanipulation of the threads.” It is the sovereign’s mostprecious faculty.\n9. Having local spies means employing the services of\nthe inhabitants of a district.\n10. Having inward spies, making use of officials of the\nenemy.\n11. Having converted spies, getting hold of the enemy’s\nspies and using them for our own purposes.\n12. Having doomed spies, doing certain things openly\nfor purposes of deception, and allowing our spies toknow of them and report them to the enemy.\n13. Surviving spies, finally, are those who bring back\nnews from the enemy’s camp.\n60\n/G53/G75/G6E/G20/G54/G7A/G75/G20/G6F/G6E/G20/G74/G68/G65/G20/G

In [64]:
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, try to make up an answer. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)


In [65]:
question = "Describe the art of war in 5 words?"
result = qa_chain({"query": question})
result["result"]

'Vital to the state, a matter of life and death. Thanks for asking!'

In [66]:
Markdown(result["result"])

Vital to the state, a matter of life and death. Thanks for asking!

In [67]:
question = "Summarize variations in tactics?"
result = qa_chain({"query": question})
Markdown(result["result"])

Tactics should be varied based on the circumstances. Thanks for asking!

In [68]:
question = "What can you derive in real modern human life from the above summary of variation in tactics?"
result = qa_chain({"query": question})
Markdown(result["result"])

From the summary of variation in tactics, we can derive the following principles that can be applied to real modern human life:

1. **Be adaptable.** The best course of action will vary depending on the circumstances. Be prepared to change your plans as needed.
2. **Don't be afraid to take risks.** Sometimes, the best way to achieve your goals is to take a risk. Don't be afraid to step outside of your comfort zone.
3. **Be creative.** There is often more than one way to achieve a goal. Be creative in your thinking and come up with new solutions to problems.
4. **Be persistent.** Don't give up easily. If you keep working at it, you will eventually achieve your goals.
5. **Be patient.** Good things take time. Don't expect to achieve your goals overnight. Be patient and persistent, and you will eventually succeed.

These principles can be applied to any area of life, from business to relationships to personal growth. By following these principles, you can increase your chances of success in whatever you do.

Thanks for asking!